## Reproduces experiment in section 2, Table 1

In [ ]:
import utils
import data_utils
import torch
from tqdm import tqdm
import pandas as pd

In [ ]:
device = "cuda"
dataset_name = "imagenet_val"
batch_size = 256
target_name = "resnet50_imagenet"
save_dir = "saved_activations"
calibration_temp = 1.1
pool_mode = "avg"

In [ ]:
model, preprocess = data_utils.get_target_model(target_name, device=device)
dataset = data_utils.get_data(dataset_name, preprocess)

In [ ]:
orig_save_path = '{}/{}_{}/orig'.format(save_dir, target_name, dataset_name)
orig_pred, orig_acc, orig_loss = utils.save_pred_acc_loss(model, dataset, device, orig_save_path, T=calibration_temp)

In [ ]:
layer_to_neuron = {'maxpool':64, 'layer1':256, 'layer2':512, 'layer3':1024, 'layer4': 2048}

In [ ]:
new_database = {'layer':[], 'unit':[]}

for cutoff in [0.0003, 0.002, 0.005, 0.02, 0.1, 0.5]:
    new_database["top_impact:{}".format(cutoff)] = []

for target_layer in layer_to_neuron.keys():
    layer_save_path = '{}/{}_{}/{}/'.format(save_dir, target_name, dataset_name, target_layer)
    summary_activations = utils.save_summary_activations(model, dataset, device, target_layer, batch_size, layer_save_path, pool_mode=pool_mode)
    for target_neuron in tqdm(range(layer_to_neuron[target_layer])):
        
        neuron_save_path = '{}/{}_{}/{}/{}'.format(save_dir, target_name, dataset_name, target_layer, target_neuron)
        new_pred, new_acc, new_loss = utils.save_pal_without_neuron(model, dataset, device, target_layer, target_neuron,
                                                                             neuron_save_path, T=calibration_temp)
        new_database['layer'].append(target_layer)
        new_database['unit'].append(target_neuron)
        neuron_impacts = utils.get_per_neuron_impact(orig_acc, orig_loss, new_acc, new_loss)
        
        sorted_act_vals, sorted_act_ids = torch.sort(summary_activations[:, target_neuron], descending=True)
        sorted_impacts = neuron_impacts.gather(0, sorted_act_ids)
        total_impact = torch.sum(torch.abs(neuron_impacts))
        cum_impact = torch.cumsum(torch.abs(sorted_impacts), dim=0)/total_impact
        for cutoff in [0.0003, 0.002, 0.005, 0.02, 0.1, 0.5]:
             new_database["top_impact:{}".format(cutoff)].append(cum_impact[int(cutoff*len(cum_impact))].item())

In [ ]:
df = pd.DataFrame(new_database)
for column in df.columns:
    if column in ['layer', 'unit']:
        continue
    print("{}, mean:{:.3f}%, std:{:.3f}%".format(column, df[column].mean()*100, df[column].std()*100))

In [ ]:
for layer in ['maxpool', 'layer1', 'layer2', 'layer3', 'layer4']:
    print(layer)
    curr_df = df[df['layer']==layer]
    for column in df.columns:
        if column in ['layer', 'unit']:
            continue
        print("{}, mean:{:.3f}%, std:{:.3f}%".format(column, curr_df[column].mean()*100, curr_df[column].std()*100))
    print("")